In [2]:
import os
import time
import pandas as pd
import numpy as np
import multiprocessing
from scipy.stats import moment
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Add, Input, Activation, concatenate
from keras.utils import to_categorical, Sequence
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from random import seed
from matplotlib import pyplot as plt
seed(0)

#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices()[1])

#from keras import backend as K
#K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


# Pruebas

## Importar datos

In [3]:
def read_txt(path):
    txt = open(path)
    data = (txt.read()).split('\n')
    txt.close()
    del txt
    return data

def sample_list(data):
    print('Generating sample list')
    sample_list = []
    for line in data:
        if ':' in line:
            sample = []
            sample_list.append(sample)
        if line != '':
            sample.append(line)
    del line
    del sample
    return sample_list

def normalize(aux):
    return (aux - np.amin(aux)) / (np.amax(aux) - np.amin(aux)) if np.amax(aux) - np.amin(aux) != 0.0 else 0.0    

def get_features(txt_sample_list):
    print('Preprocessing data')
    args = []
    label_list = []
    data = []
    cont = 0
    print('0%', end='')
    for file in txt_sample_list:
        for sample in file:
            label = np.array(process_label(sample[2].split(','))).astype('float32')
            if len(list(filter(None, sample[1].split(' ')))) == 11 and np.mean(label) != 0.0:
                args.append(sample[0])
                data.append(process_data(list(filter(None, sample[1].split(' ')))))
                #max_value = max(sample[2].split(','))
                #label.append((sample[2].split(',')).index(max_value))
                label_list.append(label)
            cont += 1
            print('\r', int(cont/(len(txt_sample_list)*len(file))*100), '%', end='')
    print()     
    return np.array(args), np.array(data), np.array(label_list)

def get_array_features(feature):
    aux = np.array(feature).astype('float32')
    aux = np.array([np.mean(aux), moment(aux, moment=2), moment(aux, moment=3), moment(aux, moment=4), moment(aux, moment=5)])
    return aux#(aux - np.amin(aux)) / (np.amax(aux) - np.amin(aux)) if np.amax(aux) - np.amin(aux) != 0.0 else 0.0

def process_label(label):
    label = np.array(label).astype('float32')
    return np.array(label)

def process_data(sample):
    data = []
    for feature in sample:
        if ',' in feature:
            for item_array in get_array_features(list(filter(None, feature.split(',')))):
                data.append(item_array)
        elif '.' in feature:
            data.append(float(feature))
        else:
            data.append(int(feature))
    data = np.array(data)
    return data#(data - np.amin(data)) / (np.amax(data) - np.amin(data)) if np.amax(data) - np.amin(data) != 0.0 else 0.0

def export_processed_data(processed_data):
    print('Exporting processed data')
    output = ''
    for sample in processed_data:
        for feature in sample:
            output += str(feature) + '\t'
        output += '\n'
        
    file = open('processed_data' + str(len(processed_data)) + '.txt','w')
    file.write(output)
    file.close()
    print('Processed data save in:', os.getcwd())

## Neural Network Model

In [4]:
def export_test_metrics(first_it, last_it, list_test_metrics, random_state, name_metrics, activation, algorithm):
    output = ''
    for name in name_metrics:
        output += str(name) + '\t'
    output += '\n'
    for metrics in list_test_metrics:
        for m in metrics:
            output += str(m) + '\t'
        output += '\n'
    file = open('test_metrics/metrics' + str(first_it) + 'to' + str(last_it) + 'random_state' + str(random_state) + 'activation' + str(activation) + str(algorithm) + '.txt','w')
    file.write(output)
    file.close()
    print('Metrics save in:', os.getcwd())

In [5]:
def export_scores(first_it, last_it, list_scores, random_state, activation, algorithm):
    output = ''
    for score in list_scores:
        output = '{\nhidden layer:[' + str(score[0]) + ', ' + str(score[1]) + ']\nscore:' + str(score[2].history) + '\n}\n'
    file = open('scores/scores' + str(first_it) + 'to' + str(last_it) + 'random_state' + str(random_state) + 'activation' + str(activation) + str(algorithm) + '.json','w')
    file.write(output)
    file.close()
    print('Scores save in:', os.getcwd())

In [6]:
'''def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))'''

def gen(x, y):
    while True:
        yield(x, y)

def neural_network_model(x, y, first_it, last_it, random_state, epochs, activation, algorithm):
    import keras
    from keras import backend as K
    from keras.models import Sequential, Model
    from keras.layers import Flatten, Dense, Add, Input, Activation, concatenate
    from keras.utils import to_categorical

    print('Processed data shape:', x.shape, end='\n\n')
    print('Label shape:', y.shape, end='\n\n')
    
    initial_random_state = -1
    second_it = -1
    for state in range(random_state + 1):
        for i in range(first_it, last_it + 1):
            for j in range(0, 101, 25):
                if j != 0:
                    if j % 25 == 0 and 'metrics' + str(i) + 'to' + str(j) + 'random_state' + str(state) + 'activation' + str(activation) + str(algorithm) + '.txt' not in os.listdir('test_metrics'):
                        first_it = i
                        second_it = j - 25 + 1 if j != 25 else 2
                        initial_random_state = state
                        break
                    else:
                        print('metrics' + str(i) + 'to' + str(j) + 'random_state' + str(state) + 'activation' + str(activation) + str(algorithm) + '.txt exist')
            else:
                continue
            break
        else:
            continue
        break

    if initial_random_state == -1 or second_it == -1:
        print('All files are generated')
        return False
        
    for split in range(initial_random_state, random_state + 1):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=split)
        
        print()
        print('x train shape:', x_train.shape)
        print('x test shape:', x_test.shape)
        print('y train shape:', y_train.shape)
        print('y test shape:', y_test.shape)
        print()

        scores = []
        test_metrics_list = []

        print('Starting Dense Neural Network')
        print('Start time:', time.asctime(time.localtime(time.time())))

        for i in range(first_it, last_it + 1):
            for j in range(second_it, 101):

                
                test_metrics = []

                print('Iteration: dense_layer_0 =', i, '\tdense_layer_1 =', j, '\trandom_state =', split, '\talgorithm =', algorithm)
                model = Sequential()
                model.add(Dense(i, input_dim=x.shape[1]))
                model.add(Activation(activation))
                model.add(Dense(j))
                model.add(Activation(activation))
                model.add(Dense(1))
                model.add(Activation('linear'))
                model.compile(loss='mae',
                              optimizer='adam',
                              metrics=['mae'])

                #model.summary()

                scores.append([i, j, model.fit_generator(generator=gen(x_train, y_train), steps_per_epoch=1, epochs=epochs, verbose=0)])

                test_metrics.append(i)
                test_metrics.append(j)
                for tm in model.evaluate(x_test, y_test):
                    test_metrics.append(tm)
                test_metrics_list.append(test_metrics)
                del test_metrics

                if j % 25 == 0:
                    export_test_metrics(i, j, test_metrics_list, split, ['hidden_layer_0', 'hidden_layer_1', 'loss_mae','mae'], activation, algorithm)
                    export_scores(i, j, scores, split, activation, algorithm)
                    del test_metrics_list
                    del scores
                    scores = []
                    test_metrics_list = []

                del model
                K.clear_session()

        print('End time:', time.asctime(time.localtime(time.time())), end='\n\n')
    return True

## Test Script

### Import data

In [7]:
with K.tf.device('/gpu:0'):
    DATADIR = "dataset/"
    txt_sample_list = []
    for name_txt in os.listdir(DATADIR):
        if ("trainingset" in name_txt and ".txt" in name_txt) and '~$' not in name_txt:
            print('Name txt:', name_txt)
            data = read_txt(DATADIR + name_txt)
            print('Reading dataset')
            txt_sample_list.append(sample_list(data))
    args, processed_data, label = get_features(txt_sample_list)
    #export_processed_data(processed_data)

Name txt: trainingset1_class.txt
Reading dataset
Generating sample list
Name txt: trainingset2_class.txt
Reading dataset
Generating sample list
Name txt: trainingset3_class.txt
Reading dataset
Generating sample list
Name txt: trainingset4_class.txt
Reading dataset
Generating sample list
Name txt: trainingset5_class.txt
Reading dataset
Generating sample list
Name txt: trainingset6_class.txt
Reading dataset
Generating sample list
Preprocessing data
 97 %%77 %78 % %
Exporting processed data
Processed data save in: C:\Users\crist\Desktop\Metasolver\extras\MCLP Neural Network


In [12]:
processed_data = processed_data.transpose()
for index in range(len(processed_data)):
    processed_data[index] = normalize(processed_data[index])
processed_data = np.array(processed_data).transpose()
export_processed_data(processed_data)

Exporting processed data
Processed data save in: C:\Users\crist\Desktop\Metasolver\extras\MCLP Neural Network


### Shape data

In [7]:
print('args shape:', args.shape)
print('X shape:', processed_data.shape)
print('y shape:', label.shape)

args shape: (9835,)
X shape: (9835, 43)
y shape: (9835, 3)


In [50]:
corr = (pd.DataFrame(pdata)).corr(method='pearson')
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

### Start Neural Network Test

In [9]:
with K.tf.device('/gpu:0'):
    neural_network_model(processed_data, label.transpose()[0], 2, 20, 5, 20, str('relu'), 'algorithm1')
    neural_network_model(processed_data, label.transpose()[1], 2, 20, 5, 20, str('relu'), 'algorithm2')
    neural_network_model(processed_data, label.transpose()[2], 2, 20, 5, 20, str('relu'), 'algorithm3')
    neural_network_model(processed_data, label.transpose()[0], 21, 40, 5, 20, str('relu'), 'algorithm1')
    neural_network_model(processed_data, label.transpose()[1], 21, 40, 5, 20, str('relu'), 'algorithm2')
    neural_network_model(processed_data, label.transpose()[2], 21, 40, 5, 20, str('relu'), 'algorithm3')
    neural_network_model(processed_data, label.transpose()[0], 41, 60, 5, 20, str('relu'), 'algorithm1')
    neural_network_model(processed_data, label.transpose()[1], 41, 60, 5, 20, str('relu'), 'algorithm2')
    neural_network_model(processed_data, label.transpose()[2], 41, 60, 5, 20, str('relu'), 'algorithm3')
    neural_network_model(processed_data, label.transpose()[0], 61, 80, 5, 20, str('relu'), 'algorithm1')
    neural_network_model(processed_data, label.transpose()[1], 61, 80, 5, 20, str('relu'), 'algorithm2')
    neural_network_model(processed_data, label.transpose()[2], 61, 80, 5, 20, str('relu'), 'algorithm3')

In [10]:
#print(scores[0].history.keys())
# "Loss"
#plt.plot(history.history['loss'])
#plt.plot(history.history['acc'])
#plt.plot(history.history['mean_squared_error'])
#plt.plot(scores[0].history['root_mean_squared_error'])
#plt.title('model loss')
#plt.ylabel('root_mean_squared_error')
#plt.xlabel('epoch')
#plt.legend(['root_mean_squared_error'], loc='upper left')
#plt.show()

### Linking test_metrics_files

In [6]:
def export_txt(data, random_state, activation, algorithm):
    output = ''
    for i in data:
        if i != '':
            output += str(i) + '\n'
    output = output.replace('.', ',')
    file = open('test_metrics/metrics_random_state' + str(random_state) + 'activation' + activation + 'algorithm' + str(algorithm) + '.txt','w')
    file.write(output)
    file.close()
    print('Metrics save in:', os.getcwd())

In [7]:
def export_excel(data, random_state, activation, algorithm):
    header = list(filter(None, data[0].replace('.', ',').split('\t')))
    del data[0]
    output = []
    for row in list(filter(None, data)):
        line = []
        for col in list(filter(None, row.replace('.', ',').split('\t'))):
            line.append(np.array(col))
        output.append(np.array(line))
        del line
    return dict(zip(header, np.array(output).transpose()))

In [13]:
'''DATADIR = "test_metrics/"
activation = 'relu'
layers = []
best_loss = 0

for algorithm in range(1, 4):
    for random_state in range(6):
        txt_sample_list = []
        list_txt = [txt for txt in os.listdir(DATADIR) if ("to" in txt and txt.endswith('random_state' + str(random_state) + 'activation' + activation + 'algorithm' + str(algorithm) + '.txt') and '~$' not in txt)]
        for name_txt in list_txt:
            data = read_txt(DATADIR + name_txt)
            header = data[0]
            data = list(filter(None, data[1:len(data)]))
            for line in data:
                if best_loss < np.array(list(filter(None, line.split('\t')))[2]).astype('float32'):
                    best = [int(line.split('\t')[0]), int(line.split('\t')[1])]
                if line != '':
                    txt_sample_list.append(line)
        txt_sample_list.insert(0, header)
        #export_txt(txt_sample_list, random_state, activation, algorithm)
        layers.append(best)
        del txt_sample_list'''

'DATADIR = "test_metrics/"\nactivation = \'relu\'\nlayers = []\nbest_loss = 0\n\nfor algorithm in range(1, 4):\n    for random_state in range(6):\n        txt_sample_list = []\n        list_txt = [txt for txt in os.listdir(DATADIR) if ("to" in txt and txt.endswith(\'random_state\' + str(random_state) + \'activation\' + activation + \'algorithm\' + str(algorithm) + \'.txt\') and \'~$\' not in txt)]\n        for name_txt in list_txt:\n            data = read_txt(DATADIR + name_txt)\n            header = data[0]\n            data = list(filter(None, data[1:len(data)]))\n            for line in data:\n                if best_loss < np.array(list(filter(None, line.split(\'\t\')))[2]).astype(\'float32\'):\n                    best = [int(line.split(\'\t\')[0]), int(line.split(\'\t\')[1])]\n                if line != \'\':\n                    txt_sample_list.append(line)\n        txt_sample_list.insert(0, header)\n        #export_txt(txt_sample_list, random_state, activation, algorithm)\n   

In [8]:
DATADIR = "test_metrics/"
activation = 'relu'
layers = []
best_loss = 0
df = []
for algorithm in range(1, 4):
    for random_state in range(6):
        txt_sample_list = []
        list_txt = [txt for txt in os.listdir(DATADIR) if ("to" in txt and txt.endswith('random_state' + str(random_state) + 'activation' + activation + 'algorithm' + str(algorithm) + '.txt') and '~$' not in txt)]
        for name_txt in list_txt:
            data = read_txt(DATADIR + name_txt)
            header = data[0]
            data = list(filter(None, data[1:len(data)]))
            for line in data:
                if best_loss < np.array(list(filter(None, line.split('\t')))[2]).astype('float32'):
                    best = [int(line.split('\t')[0]), int(line.split('\t')[1])]
                if line != '':
                    txt_sample_list.append(line)
        txt_sample_list.insert(0, header)
        df.append(['metrics_randst' + str(random_state) + '_act' + activation + '_alg' + str(algorithm), pd.DataFrame(export_excel(txt_sample_list, random_state, activation, algorithm))])
        layers.append(best)
        del txt_sample_list
with pd.ExcelWriter('test_metrics/test_metrics.xlsx') as writer:
    for dataframe in df:
        dataframe[1].to_excel(writer, sheet_name=dataframe[0], index=False)
print(layers)

[[9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75], [9, 75]]


# Definitive Nerual Network Model

### Start Neural Network with parameters

In [15]:
def neural_network_model(x, y, dense1, dense2, epochs):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=800, random_state=3)

    #print(x_train.shape)
    #print(x_test.shape)
    #print(y_train.shape)
    #print(y_test.shape)

    print('Starting Dense Neural Network')
    print('Start time:', time.asctime(time.localtime(time.time())))

    model = Sequential()
    model.add(Dense(dense1, input_dim=x.shape[1]))
    model.add(Activation('relu'))
    model.add(Dense(dense2))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('linear'))
    model.compile(loss='mae',
                  optimizer='adam',
                  metrics=['mae'])

    model.summary()

    scores = model.fit_generator(generator=gen(x_train, y_train), steps_per_epoch=1, epochs=epochs, verbose=0)
    test_metrics = model.evaluate(x_test, y_test)
    #export_test_metrics(first_it, last_it, test_metrics, ['loss_msre', 'accuracy', 'mse', 'root_mean_squared_error','mae'])
    #export_scores(first_it, last_it, scores)
    print('End time:', time.asctime(time.localtime(time.time())), end='\n\n')
    print('loss:', test_metrics[0])
    print('metric:', test_metrics[1])

    return model, x_test, y_test

with K.tf.device('/gpu:0'):
    '''DATADIR = "dataset/"
    txt_sample_list = []
    for name_txt in os.listdir(DATADIR):
        if ("new_dataset" in name_txt and ".txt" in name_txt) and '~$' not in name_txt:
            print('Name txt:', name_txt)
            data = read_txt(DATADIR + name_txt)
            txt_sample_list.append(sample_list(data))
    args, processed_data, label = get_features(txt_sample_list)
    export_processed_data(processed_data)'''
    
    a1_model, a1_x_test, a1_y_test = neural_network_model(processed_data, label.transpose()[0], best[0], best[1], 20)
    a2_model, a2_x_test, a2_y_test = neural_network_model(processed_data, label.transpose()[1], best[0], best[1], 20)
    a3_model, a3_x_test, a3_y_test = neural_network_model(processed_data, label.transpose()[2], best[0], best[1], 20)
    
print(a1_y_test.shape)
print(a2_y_test.shape)
print(a3_y_test.shape)

predictions = np.concatenate([[a1_model.predict(a1_x_test).ravel(),
                              a2_model.predict(a2_x_test).ravel(),
                              a3_model.predict(a3_x_test).ravel()]])

Starting Dense Neural Network
Start time: Sun Dec  8 22:11:09 2019
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 9)                 396       
_________________________________________________________________
activation_1 (Activation)    (None, 9)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 75)                750       
_________________________________________________________________
activation_2 (Activation)    (None, 75)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 76        
_________________________________________________________________
activation_3 (Activation)    (None, 1)                 0         
Tot

In [35]:
max_values = [np.max(predictions.transpose()[i]) for i in range(len(predictions.transpose()))]
max_values = np.array(max_values)

fail = []
for i in range(len(max_values)):
    if list(predictions.transpose()[i]).index(np.max(predictions.transpose()[i])) != list(np.array([a1_y_test[i], a2_y_test[i], a3_y_test[i]])).index(np.max([a1_y_test[i], a2_y_test[i], a3_y_test[i]])):
        fail.append(False)
    else:
        fail.append(True)
fail = np.array(fail)

#m = [(max_values[i], predictions.transpose()[i], np.array([a1_y_test[i], a2_y_test[i], a3_y_test[i]])) for i in range(len(max_values))]
m = [(max_values[i],
      predictions.transpose()[i],
      np.array([a1_y_test[i], a2_y_test[i], a3_y_test[i]]),
      list(predictions.transpose()[i]).index(np.max(predictions.transpose()[i])),
      list(np.array([a1_y_test[i], a2_y_test[i], a3_y_test[i]])).index(np.max([a1_y_test[i], a2_y_test[i], a3_y_test[i]])) + 1,
      fail[i])
      for i in range(len(max_values))]
    
results = pd.DataFrame(m, columns=['max_value', 'prediction_array', 'test_array', 'prediction', 'test', 'correct?'])
results.to_excel('test_data.xls', index=False)

In [36]:
results

,max_value,prediction_array,test_array,prediction,test,correct?
0,0.277167,"[0.1744271, 0.0520385, 0.2771668]","[0.71390456, 1.0, 0.0]",2,2,False
1,0.256885,"[0.16857813, 0.052912638, 0.2568855]","[0.7253402, 0.0, 1.0]",2,3,True
2,0.261822,"[0.17552936, 0.05791498, 0.26182207]","[0.0, 0.5816468, 1.0]",2,3,True
3,0.273811,"[0.17515779, 0.050991863, 0.27381077]","[1.0, 0.72691303, 0.0]",2,1,False
4,0.258209,"[0.1700222, 0.049047627, 0.2582089]","[0.7043222, 0.0, 1.0]",2,3,True
5,0.276335,"[0.17442887, 0.049248464, 0.27633503]","[0.0, 1.0, 0.29787233]",2,2,False
6,0.241741,"[0.16651575, 0.037225764, 0.24174085]","[1.0, 0.0, 0.4668734]",2,1,False
7,0.275104,"[0.17504576, 0.050529413, 0.27510363]","[1.0, 0.0, 0.7791781]",2,1,False
8,0.242344,"[0.1721188, 0.06896472, 0.24234386]","[0.0, 0.4595724, 1.0]",2,3,True
9,0.277992,"[0.17436177, 0.04927575, 0.27799222]","[0.045996193, 1.0, 0.0]",2,2,False


# De aquí en adelante no tomar en cuenta !!!

In [18]:
'''def discriminator(X):
    input_shape = Input(shape=(X, ))
    prediction = Dense(256, activation='relu')(input_shape)
    prediction = Dense(128, activation='relu')(prediction)
    prediction = Dense(3, activation = 'softmax')(prediction)
    return Model(input_shape, prediction, name='Discriminator')'''

"def discriminator(X):\n    input_shape = Input(shape=(X, ))\n    prediction = Dense(256, activation='relu')(input_shape)\n    prediction = Dense(128, activation='relu')(prediction)\n    prediction = Dense(3, activation = 'softmax')(prediction)\n    return Model(input_shape, prediction, name='Discriminator')"

In [19]:
'''def generator(z):
    input_shape = Input(shape=(z, ))
    prediction = Dense(256, activation='relu')(input_shape)
    prediction = Dense(128, activation='relu')(prediction)
    prediction = Dense(43, activation = 'relu')(prediction)
    return Model(input_shape, prediction, name='Generator')'''

"def generator(z):\n    input_shape = Input(shape=(z, ))\n    prediction = Dense(256, activation='relu')(input_shape)\n    prediction = Dense(128, activation='relu')(prediction)\n    prediction = Dense(43, activation = 'relu')(prediction)\n    return Model(input_shape, prediction, name='Generator')"

In [20]:
'''from keras.utils import plot_model

z = 100
data_size = x_train.shape[1]
batch_size = 100

G = generator(z)
G.compile(loss=['categorical_crossentropy'],
          optimizer='adam',
          metrics=['accuracy'])
#G.summary()

D = discriminator(data_size)
D.compile(loss=['categorical_crossentropy'],
          optimizer='adam')
#D.summary()

input_shape = Input(shape=(z, ))
fake_data = G(input_shape)
D.trainable = False
valid = D(fake_data)

GAN = Model(input_shape, valid, name='GAN')
GAN.name = 'GAN'
GAN.compile(loss=['categorical_crossentropy'],
            optimizer='adam')

plot_model(G, to_file='Generator.png')
plot_model(D, to_file='Discriminator.png')
plot_model(GAN, to_file='GAN.png')

d_loss_real = D.fit(x_train, y_train).history['loss']
trained = []
epochs = 1000
for epoch in range(epochs + 1):
    idx = np.random.randint(0, x_train.shape[0], int(batch_size/2))
    imgs = x_train[idx]
    
    
    
    
    print('epoch', epoch)
    noise = np.random.uniform(0, 1, size=(batch_size, z))
    noise_y = to_categorical(np.random.randint(0, 3, size=(batch_size, )), num_classes = 3)
    print(noise.shape)
    print(noise_y.shape)
    
    generated_data = G.predict(noise)
    
    #d_loss_real = D.train_on_batch(x_train[idx], y_train[idx])
    d_loss_fake = D.train_on_batch(generated_data, noise_y)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    
    noise = np.random.uniform(0, 1, size=(batch_size, z))
    noise_y = to_categorical(np.random.randint(0, 3, size=(batch_size, )), num_classes = 3)

    valid = []
    for sample in D.predict(generated_data):
        classes = []
        for num_class in sample:
            if num_class == np.max(sample):
                classes.append(1)
            else:
                classes.append(0)
        valid.append(np.array(classes))
        del classes
    valid = np.array(valid)
    #print(valid)
    
    
    g_loss = GAN.train_on_batch(noise, valid)
    print (epoch, '[D loss:', d_loss, ', acc.:', '100*d_loss[1]', '] [G loss:', g_loss)

noise = np.random.uniform(0, 1, size=(batch_size, z))
predict = G.predict(noise)
#print(predict)
#print(D.predict(predict))'''

"from keras.utils import plot_model\n\nz = 100\ndata_size = x_train.shape[1]\nbatch_size = 100\n\nG = generator(z)\nG.compile(loss=['categorical_crossentropy'],\n          optimizer='adam',\n          metrics=['accuracy'])\n#G.summary()\n\nD = discriminator(data_size)\nD.compile(loss=['categorical_crossentropy'],\n          optimizer='adam')\n#D.summary()\n\ninput_shape = Input(shape=(z, ))\nfake_data = G(input_shape)\nD.trainable = False\nvalid = D(fake_data)\n\nGAN = Model(input_shape, valid, name='GAN')\nGAN.name = 'GAN'\nGAN.compile(loss=['categorical_crossentropy'],\n            optimizer='adam')\n\nplot_model(G, to_file='Generator.png')\nplot_model(D, to_file='Discriminator.png')\nplot_model(GAN, to_file='GAN.png')\n\nd_loss_real = D.fit(x_train, y_train).history['loss']\ntrained = []\nepochs = 1000\nfor epoch in range(epochs + 1):\n    idx = np.random.randint(0, x_train.shape[0], int(batch_size/2))\n    imgs = x_train[idx]\n    \n    \n    \n    \n    print('epoch', epoch)\n    

In [21]:
'''print(predict)
print(D.predict(predict))
print(G.metrics_names)
print(g_loss)'''

'print(predict)\nprint(D.predict(predict))\nprint(G.metrics_names)\nprint(g_loss)'

In [22]:
'''def discriminator():
    input_shape = Input(shape=(43, ))
    prediction = Dense(256, activation='relu')(input_shape)
    prediction = Dense(128, activation='relu')(prediction)
    prediction_1 = Dense(1, activation='softmax')(prediction)
    prediction_2 = Dense(100, activation='sigmoid')(prediction)
    return Model(input_shape, [prediction_1, prediction_2])

D = discriminator()
D.trainable = False
D.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
D.summary()'''

"def discriminator():\n    input_shape = Input(shape=(43, ))\n    prediction = Dense(256, activation='relu')(input_shape)\n    prediction = Dense(128, activation='relu')(prediction)\n    prediction_1 = Dense(1, activation='softmax')(prediction)\n    prediction_2 = Dense(100, activation='sigmoid')(prediction)\n    return Model(input_shape, [prediction_1, prediction_2])\n\nD = discriminator()\nD.trainable = False\nD.compile(loss='binary_crossentropy',\n              optimizer='adam',\n              metrics=['accuracy'])\nD.summary()"

In [23]:
'''def generator():
    x = Input(shape=(100, ))
    y = Input(shape=(1, ))
    input_shape = concatenate([x, y])
    prediction = Dense(256, activation='relu')(input_shape)
    prediction = Dense(128, activation='relu')(prediction)
    prediction = Dense(43, activation='relu')(prediction)
    
    return Model([x, y], prediction)

G = generator()'''

"def generator():\n    x = Input(shape=(100, ))\n    y = Input(shape=(1, ))\n    input_shape = concatenate([x, y])\n    prediction = Dense(256, activation='relu')(input_shape)\n    prediction = Dense(128, activation='relu')(prediction)\n    prediction = Dense(43, activation='relu')(prediction)\n    \n    return Model([x, y], prediction)\n\nG = generator()"

In [24]:
'''def gan(generator, discriminator):
    z_x = Input(shape=(100,))
    z_y = Input(shape=(1, ))

    generated = generator([z_x, z_y])
    pred_dis, pred_gen = discriminator(generated)
    return Model([z_x, z_y], [pred_gen, pred_dis])

GAN = gan(G, D)
GAN.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'],
            optimizer='adam',
            metrics=['accuracy'])'''

"def gan(generator, discriminator):\n    z_x = Input(shape=(100,))\n    z_y = Input(shape=(1, ))\n\n    generated = generator([z_x, z_y])\n    pred_dis, pred_gen = discriminator(generated)\n    return Model([z_x, z_y], [pred_gen, pred_dis])\n\nGAN = gan(G, D)\nGAN.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'],\n            optimizer='adam',\n            metrics=['accuracy'])"

In [25]:
'''from keras.utils import plot_model
plot_model(G, to_file='Generator.png')
plot_model(D, to_file='Discriminator.png')
plot_model(GAN, to_file='GAN.png')'''

"from keras.utils import plot_model\nplot_model(G, to_file='Generator.png')\nplot_model(D, to_file='Discriminator.png')\nplot_model(GAN, to_file='GAN.png')"

In [26]:
'''with K.tf.device('/gpu:0'):
    z = 100

    scores = []
    test_metrics = []
    epochs = 1000
    batch_size = 100
    for epoch in range(0, epochs + 1):
        print('Starting epoch', epoch)
        for batch in range(0, len(x_train) - batch_size, batch_size):
            noise_x = np.random.uniform(0, 1, size=(batch_size, z)).astype('float32')
            noise_y = np.random.randint(0, 3, size=(z, )).astype('float32')
            generated = G.predict([noise_x, noise_y])
            #print(generated.shape)
            #print(x_train[batch:batch + batch_size].shape)

            batch_x = x_train[batch:batch + batch_size]
            batch_y = y_train[batch:batch + batch_size]
            pred_dis, pred_gen = D.predict(batch_x)
            #print(pred_dis.shape)
            #print(pred_gen.shape)

            train = D.fit(batch_x, [batch_y, pred_gen])
            pred_dis, pred_gen = D.predict(generated)
            #print(pred_dis.shape)
            #print(pred_gen.shape)

            noise = np.random.uniform(0, 1, size=(batch_size, z)).astype('float32')
            scores.append(GAN.fit([noise, batch_y], [pred_gen, batch_y]))
            test_metrics.append(D.evaluate(x_test, [y_test, pred_gen]))'''

"with K.tf.device('/gpu:0'):\n    z = 100\n\n    scores = []\n    test_metrics = []\n    epochs = 1000\n    batch_size = 100\n    for epoch in range(0, epochs + 1):\n        print('Starting epoch', epoch)\n        for batch in range(0, len(x_train) - batch_size, batch_size):\n            noise_x = np.random.uniform(0, 1, size=(batch_size, z)).astype('float32')\n            noise_y = np.random.randint(0, 3, size=(z, )).astype('float32')\n            generated = G.predict([noise_x, noise_y])\n            #print(generated.shape)\n            #print(x_train[batch:batch + batch_size].shape)\n\n            batch_x = x_train[batch:batch + batch_size]\n            batch_y = y_train[batch:batch + batch_size]\n            pred_dis, pred_gen = D.predict(batch_x)\n            #print(pred_dis.shape)\n            #print(pred_gen.shape)\n\n            train = D.fit(batch_x, [batch_y, pred_gen])\n            pred_dis, pred_gen = D.predict(generated)\n            #print(pred_dis.shape)\n            #pr

In [27]:
'''def export_test_metrics(list_test_metrics):
    output = ''
    name_metrics = ['loss', 'loss_batch_y', 'loss_generated_data', 'batch_y_acc', 'generated_data_acc']
    for name in name_metrics:
        output += str(name) + '\t'
    output += '\n'
    for metrics in list_test_metrics:
        for m in metrics:
            output += str(m) + '\t'
        output += '\n'
    file = open('metrics.txt','w')
    file.write(output)
    file.close()
    print('Metrics save in:', os.getcwd())

export_test_metrics(test_metrics)'''

"def export_test_metrics(list_test_metrics):\n    output = ''\n    name_metrics = ['loss', 'loss_batch_y', 'loss_generated_data', 'batch_y_acc', 'generated_data_acc']\n    for name in name_metrics:\n        output += str(name) + '\t'\n    output += '\n'\n    for metrics in list_test_metrics:\n        for m in metrics:\n            output += str(m) + '\t'\n        output += '\n'\n    file = open('metrics.txt','w')\n    file.write(output)\n    file.close()\n    print('Metrics save in:', os.getcwd())\n\nexport_test_metrics(test_metrics)"

In [28]:
'''print(scores[0].history)'''

'print(scores[0].history)'

In [29]:
'''import json

def export_scores(list_scores):
    output = []
    for s in scores:
        output.append(dict(s.history))
    file = open('scores.json','w')
    file.write(json.dumps(output))
    file.close()
    print('Scores save in:', os.getcwd())
    
export_scores(scores)'''

"import json\n\ndef export_scores(list_scores):\n    output = []\n    for s in scores:\n        output.append(dict(s.history))\n    file = open('scores.json','w')\n    file.write(json.dumps(output))\n    file.close()\n    print('Scores save in:', os.getcwd())\n    \nexport_scores(scores)"

In [30]:
'''def read_json():
    file = open('scores.json')
    json_data = file.read()
    file.close()
    return json.loads(json_data)

json_data = read_json()'''

"def read_json():\n    file = open('scores.json')\n    json_data = file.read()\n    file.close()\n    return json.loads(json_data)\n\njson_data = read_json()"